# 🚀 TurboBatch Tutorial: High-Performance Transformer Inference

Welcome to the **TurboBatch** tutorial! This notebook will guide you through using TurboBatch to achieve **10x faster** transformer inference through intelligent dynamic batching.

## 📖 What You'll Learn:
- ⚡ How to set up TurboBatch for your models
- 🎯 Basic and advanced usage patterns  
- 📊 Performance comparisons and benchmarks
- 🔧 Optimization techniques and best practices
- 🚀 Real-world examples and use cases

## 🎯 Prerequisites:
- Basic Python knowledge
- Familiarity with Transformers library
- Understanding of NLP concepts (helpful but not required)

Let's get started! 🌟

## 📦 Installation and Setup

First, let's install the required dependencies:

In [ ]:
# Install required packages (uncomment if needed)
# %pip install torch transformers numpy matplotlib seaborn tqdm turbobatch

# Import necessary libraries
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict

# Setup plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All packages installed successfully!")
print(f"🐍 Python version: {sys.version}")
print(f"📊 NumPy version: {np.__version__}")

In [ ]:
# Import TurboBatch (assuming it's in the parent directory or installed via pip)
sys.path.append('..')
from turbobatch import TurboBatcher, PredictionResult, BatchStats
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print("✅ TurboBatch imported successfully!")
print("🚀 Ready to accelerate your transformer inference!")

## 🏃‍♂️ Quick Start: Your First Dynamic Batch

Let's start with a simple sentiment analysis example:

In [ ]:
# Load a pre-trained sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print(f"📥 Loaded model: {model_name}")
print(f"🏷️  Model type: {type(model).__name__}")
print(f"📊 Model parameters: {model.num_parameters():,}")

In [ ]:
# Create a TurboBatcher instance
batcher = TurboBatcher(
    model=model,
    tokenizer=tokenizer,
    max_batch_size=16,
    timeout_ms=100,
    adaptive_batching=True,
    performance_monitoring=True
)

print("🚀 TurboBatcher created with the following configuration:")
print(f"   📦 Max batch size: {batcher.max_batch_size}")
print(f"   ⏱️  Timeout: {batcher.timeout_ms}ms")
print(f"   🧠 Adaptive batching: {batcher.adaptive_batching}")
print(f"   📈 Performance monitoring: {batcher.performance_monitoring}")

In [ ]:
# Prepare some test data
test_texts = [
    "I absolutely love this product! It's amazing!",
    "This is the worst thing I've ever bought.",
    "It's okay, nothing special but does the job.",
    "Outstanding quality and excellent customer service!",
    "Poor build quality, would not recommend.",
    "Great value for money, very satisfied!",
    "Disappointing experience overall.",
    "Perfect! Exactly what I was looking for.",
    "Could be better but it works fine.",
    "Incredible performance, highly recommended!"
]

print(f"📝 Prepared {len(test_texts)} test texts for prediction")

# Quick single prediction
print("\n🔍 Single prediction example:")
start_time = time.time()
predictions = batcher.predict(test_texts)
end_time = time.time()

print(f"✅ Completed {len(predictions)} predictions in {end_time - start_time:.3f} seconds")
print(f"? Throughput: {len(predictions) / (end_time - start_time):.2f} predictions/second")

# Display results
print("\n📊 Prediction Results:")
for i, (text, pred) in enumerate(zip(test_texts, predictions)):
    sentiment = "POSITIVE 😊" if pred.label == 1 else "NEGATIVE 😞"
    print(f"{i+1:2d}. {sentiment} ({pred.score:.3f}) | {text[:50]}...")

In [ ]:
# Example: Creating a sentiment analysis API wrapper
class SentimentAPI:
    """Simple API wrapper using TurboBatcher for efficient sentiment analysis"""
    
    def __init__(self, model, tokenizer, max_batch_size=32):
        self.batcher = TurboBatcher(
            model=model,
            tokenizer=tokenizer,
            max_batch_size=max_batch_size,
            adaptive_batching=True,
            performance_monitoring=True
        )
        
    def analyze_sentiment(self, text):
        """Analyze sentiment for a single text"""
        result = self.batcher.predict(text)
        return {
            'text': text,
            'sentiment': 'POSITIVE' if result.label == 1 else 'NEGATIVE',
            'confidence': result.score,
            'processing_time': result.processing_time
        }
    
    def batch_analyze(self, texts):
        """Analyze sentiment for multiple texts efficiently"""
        results = self.batcher.predict(texts)
        return [
            {
                'text': text,
                'sentiment': 'POSITIVE' if result.label == 1 else 'NEGATIVE',
                'confidence': result.score,
                'processing_time': result.processing_time
            }
            for text, result in zip(texts, results)
        ]

# Create API instance
api = SentimentAPI(model, tokenizer)

# Test single analysis
sample_text = "This product exceeded my expectations!"
result = api.analyze_sentiment(sample_text)
print("📝 Single Analysis Result:")
print(f"Text: {result['text']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Confidence: {result['confidence']:.3f}")

# Test batch analysis
batch_results = api.batch_analyze(test_texts[:5])
print(f"\n📊 Batch Analysis Results ({len(batch_results)} texts):")
for i, result in enumerate(batch_results):
    print(f"{i+1}. {result['sentiment']:8s} ({result['confidence']:.3f}) | {result['text'][:40]}...")

## 📊 Performance Comparison: Batched vs Individual Processing

Let's see the power of dynamic batching in action!

In [ ]:
def generate_test_data(num_samples: int) -> List[str]:
    """Generate diverse test data for benchmarking"""
    base_texts = [
        "I love this amazing product!",
        "This is absolutely terrible.",
        "The weather is quite nice today.",
        "I'm feeling neutral about this situation.",
        "Fantastic results! Highly recommended!",
        "Poor quality, very disappointed.",
        "It's okay, meets basic requirements.",
        "Excellent value for money!",
        "Could be improved but not bad.",
        "Perfect solution for my needs!"
    ]
    
    return [base_texts[i % len(base_texts)] + f" (Sample {i+1})" for i in range(num_samples)]

# Generate test data
test_data = generate_test_data(100)
print(f"📊 Generated {len(test_data)} test samples for benchmarking")

In [ ]:
# Benchmark individual vs batched processing
def benchmark_individual_processing(texts: List[str]):
    """Benchmark individual processing (one by one)"""
    individual_batcher = DynamicBatcher(
        model=model,
        tokenizer=tokenizer,
        max_batch_size=1,  # Force individual processing
        timeout_ms=0,
        adaptive_batching=False
    )
    
    start_time = time.time()
    predictions = individual_batcher.predict(texts)
    end_time = time.time()
    
    return end_time - start_time, predictions

def benchmark_batched_processing(texts: List[str], batch_size: int = 16):
    """Benchmark batched processing"""
    batched_batcher = DynamicBatcher(
        model=model,
        tokenizer=tokenizer,
        max_batch_size=batch_size,
        timeout_ms=50,
        adaptive_batching=True
    )
    
    start_time = time.time()
    predictions = batched_batcher.predict(texts)
    end_time = time.time()
    
    return end_time - start_time, predictions

# Run benchmarks
print("🏃‍♂️ Running individual processing benchmark...")
individual_time, individual_preds = benchmark_individual_processing(test_data)

print("🚀 Running batched processing benchmark...")
batched_time, batched_preds = benchmark_batched_processing(test_data)

# Calculate speedup
speedup = individual_time / batched_time

print("\n📊 Benchmark Results:")
print("-" * 50)
print(f"Individual Processing: {individual_time:.3f}s")
print(f"Batched Processing:    {batched_time:.3f}s")
print(f"Speedup:              {speedup:.2f}x faster! 🚀")
print(f"Throughput Individual: {len(test_data)/individual_time:.2f} samples/sec")
print(f"Throughput Batched:    {len(test_data)/batched_time:.2f} samples/sec")

In [ ]:
# Visualize the performance comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Processing time comparison
methods = ['Individual\nProcessing', 'Dynamic\nBatching']
times = [individual_time, batched_time]
colors = ['#ff7f7f', '#7fbf7f']

bars1 = ax1.bar(methods, times, color=colors, alpha=0.8)
ax1.set_ylabel('Processing Time (seconds)')
ax1.set_title('⏱️ Processing Time Comparison')
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, time_val in zip(bars1, times):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{time_val:.3f}s',
             ha='center', va='bottom', fontweight='bold')

# Throughput comparison
throughputs = [len(test_data)/individual_time, len(test_data)/batched_time]
bars2 = ax2.bar(methods, throughputs, color=colors, alpha=0.8)
ax2.set_ylabel('Throughput (samples/second)')
ax2.set_title('🚀 Throughput Comparison')
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar, throughput in zip(bars2, throughputs):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{throughput:.1f}',
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n🎉 Dynamic Batching is {speedup:.2f}x faster than individual processing!")

## 🎯 Batch Size Optimization

Let's find the optimal batch size for maximum performance:

In [ ]:
# Test different batch sizes
batch_sizes = [1, 2, 4, 8, 16, 32, 64]
performance_results = {}

print("🔍 Testing different batch sizes...")
for batch_size in batch_sizes:
    print(f"  Testing batch size: {batch_size}")
    
    # Create batcher with specific batch size
    test_batcher = TurboBatcher(
        model=model,
        tokenizer=tokenizer,
        max_batch_size=batch_size,
        timeout_ms=50,
        adaptive_batching=False  # Fixed batch size for this test
    )
    
    # Warm up
    _ = test_batcher.predict(test_data[:5])
    
    # Benchmark
    start_time = time.time()
    predictions = test_batcher.predict(test_data)
    end_time = time.time()
    
    processing_time = end_time - start_time
    throughput = len(test_data) / processing_time
    
    performance_results[batch_size] = {
        'time': processing_time,
        'throughput': throughput
    }
    
    print(f"    ⏱️  Time: {processing_time:.3f}s | 🚀 Throughput: {throughput:.2f} samples/sec")

print("\n✅ Batch size optimization completed!")

In [ ]:
# Visualize batch size optimization results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

batch_sizes_list = list(performance_results.keys())
times = [performance_results[bs]['time'] for bs in batch_sizes_list]
throughputs = [performance_results[bs]['throughput'] for bs in batch_sizes_list]

# Processing time vs batch size
ax1.plot(batch_sizes_list, times, marker='o', linewidth=2, markersize=8, color='#e74c3c')
ax1.set_xlabel('Batch Size')
ax1.set_ylabel('Processing Time (seconds)')
ax1.set_title('⏱️ Processing Time vs Batch Size')
ax1.grid(True, alpha=0.3)
ax1.set_xscale('log', base=2)

# Throughput vs batch size
ax2.plot(batch_sizes_list, throughputs, marker='s', linewidth=2, markersize=8, color='#27ae60')
ax2.set_xlabel('Batch Size')
ax2.set_ylabel('Throughput (samples/second)')
ax2.set_title('🚀 Throughput vs Batch Size')
ax2.grid(True, alpha=0.3)
ax2.set_xscale('log', base=2)

# Find optimal batch size
optimal_batch_size = max(performance_results.keys(), key=lambda x: performance_results[x]['throughput'])
optimal_throughput = performance_results[optimal_batch_size]['throughput']

# Mark optimal point
ax2.scatter([optimal_batch_size], [optimal_throughput], 
           color='red', s=100, zorder=5, label=f'Optimal: {optimal_batch_size}')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"\n🎯 Optimal batch size: {optimal_batch_size}")
print(f"🚀 Maximum throughput: {optimal_throughput:.2f} samples/second")

## 🧠 Adaptive Batching in Action

Watch how TurboBatcher automatically adapts to different workloads:

In [ ]:
# Create an adaptive batcher
adaptive_batcher = TurboBatcher(
    model=model,
    tokenizer=tokenizer,
    max_batch_size=32,
    timeout_ms=100,
    adaptive_batching=True,
    performance_monitoring=True
)

# Simulate different workload scenarios
scenarios = [
    ("🐌 Light Load", generate_test_data(5)),
    ("📈 Medium Load", generate_test_data(25)),
    ("🚀 Heavy Load", generate_test_data(100)),
    ("💥 Burst Load", generate_test_data(200)),
]

adaptive_results = []

print("🧠 Testing Adaptive Batching with different workloads...")
print("-" * 60)

for scenario_name, test_data in scenarios:
    print(f"\n{scenario_name} - {len(test_data)} samples:")
    
    # Reset performance stats
    adaptive_batcher._reset_stats() if hasattr(adaptive_batcher, '_reset_stats') else None
    
    # Process with timing
    start_time = time.time()
    predictions = adaptive_batcher.predict(test_data)
    end_time = time.time()
    
    # Get performance stats
    stats = adaptive_batcher.get_performance_stats()
    
    processing_time = end_time - start_time
    throughput = len(test_data) / processing_time
    
    result = {
        'scenario': scenario_name,
        'samples': len(test_data),
        'time': processing_time,
        'throughput': throughput,
        'avg_batch_size': stats.get('avg_batch_size', 'N/A'),
        'total_batches': stats.get('total_batches', 'N/A')
    }
    
    adaptive_results.append(result)
    
    print(f"  ⏱️  Time: {processing_time:.3f}s")
    print(f"  🚀 Throughput: {throughput:.2f} samples/sec")
    print(f"  ? Avg batch size: {result['avg_batch_size']}")
    print(f"  ? Total batches: {result['total_batches']}")

print(f"\n✅ Adaptive batching test completed!")
print("📊 Notice how TurboBatcher adapts batch sizes based on workload!")

In [ ]:
# Visualize adaptive batching results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🧠 Adaptive Batching Performance Analysis', fontsize=16, fontweight='bold')

scenarios_names = [r['scenario'] for r in adaptive_results]
sample_counts = [r['samples'] for r in adaptive_results]
throughputs = [r['throughput'] for r in adaptive_results]
times = [r['time'] for r in adaptive_results]

# Throughput by scenario
bars1 = ax1.bar(range(len(scenarios_names)), throughputs, 
                color=['#3498db', '#e67e22', '#e74c3c', '#9b59b6'], alpha=0.8)
ax1.set_xticks(range(len(scenarios_names)))
ax1.set_xticklabels([s.split()[1] for s in scenarios_names], rotation=45)
ax1.set_ylabel('Throughput (samples/sec)')
ax1.set_title('🚀 Throughput by Workload')
ax1.grid(True, alpha=0.3)

# Processing time by scenario
bars2 = ax2.bar(range(len(scenarios_names)), times,
                color=['#3498db', '#e67e22', '#e74c3c', '#9b59b6'], alpha=0.8)
ax2.set_xticks(range(len(scenarios_names)))
ax2.set_xticklabels([s.split()[1] for s in scenarios_names], rotation=45)
ax2.set_ylabel('Processing Time (seconds)')
ax2.set_title('⏱️ Processing Time by Workload')
ax2.grid(True, alpha=0.3)

# Efficiency (throughput per sample)
efficiency = [t/s for t, s in zip(throughputs, sample_counts)]
bars3 = ax3.bar(range(len(scenarios_names)), efficiency,
                color=['#3498db', '#e67e22', '#e74c3c', '#9b59b6'], alpha=0.8)
ax3.set_xticks(range(len(scenarios_names)))
ax3.set_xticklabels([s.split()[1] for s in scenarios_names], rotation=45)
ax3.set_ylabel('Efficiency (throughput/sample)')
ax3.set_title('⚡ Processing Efficiency')
ax3.grid(True, alpha=0.3)

# Scalability chart
ax4.scatter(sample_counts, throughputs, s=100, 
           c=['#3498db', '#e67e22', '#e74c3c', '#9b59b6'], alpha=0.8)
ax4.set_xlabel('Number of Samples')
ax4.set_ylabel('Throughput (samples/sec)')
ax4.set_title('📈 Scalability Analysis')
ax4.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(sample_counts, throughputs, 1)
p = np.poly1d(z)
ax4.plot(sample_counts, p(sample_counts), "--", alpha=0.7, color='gray')

plt.tight_layout()
plt.show()

print("📊 Adaptive batching automatically optimizes performance based on workload!")

## 🎯 Real-World Use Case: Sentiment Analysis API

Let's simulate a real-world scenario where you're building a sentiment analysis API:

In [ ]:
import random
import threading
import queue
from concurrent.futures import ThreadPoolExecutor
import time

class SentimentAPI:
    """Simulated sentiment analysis API using TurboBatcher"""
    
    def __init__(self):
        self.batcher = TurboBatcher(
            model=model,
            tokenizer=tokenizer,
            max_batch_size=32,
            timeout_ms=50,
            adaptive_batching=True,
            performance_monitoring=True
        )
        self.request_count = 0
        self.total_processing_time = 0
    
    def analyze_sentiment(self, text: str) -> dict:
        """Analyze sentiment of a single text"""
        start_time = time.time()
        
        # Use TurboBatcher for efficient processing
        result = self.batcher.predict([text])[0]
        
        end_time = time.time()
        
        self.request_count += 1
        self.total_processing_time += (end_time - start_time)
        
        return {
            'text': text,
            'sentiment': 'POSITIVE' if result.label == 1 else 'NEGATIVE',
            'confidence': result.score,
            'processing_time': end_time - start_time
        }
    
    def get_stats(self) -> dict:
        """Get API performance statistics"""
        avg_response_time = self.total_processing_time / self.request_count if self.request_count > 0 else 0
        batcher_stats = self.batcher.get_performance_stats()
        
        return {
            'total_requests': self.request_count,
            'avg_response_time': avg_response_time,
            'total_processing_time': self.total_processing_time,
            'batcher_stats': batcher_stats
        }

# Create API instance
api = SentimentAPI()
print("🌐 Sentiment Analysis API created with TurboBatcher backend!")

In [ ]:
# Simulate realistic API requests
sample_reviews = [
    "This product is absolutely amazing! I love it!",
    "Worst purchase ever. Complete waste of money.",
    "Good quality, fast delivery. Satisfied with the purchase.",
    "The item arrived damaged and customer service was unhelpful.",
    "Excellent value for money. Highly recommend!",
    "Product doesn't match the description. Very disappointed.",
    "Fast shipping, great packaging. Product works perfectly.",
    "Poor quality materials. Broke after just one week.",
    "Outstanding customer service and premium quality product.",
    "Overpriced for what you get. Not worth the money."
]

def simulate_user_request():
    """Simulate a user making an API request"""
    review = random.choice(sample_reviews)
    # Add some randomness to make it more realistic
    review += f" (User {random.randint(1, 1000)})"
    
    return api.analyze_sentiment(review)

# Simulate concurrent API requests
print("🚀 Simulating 100 concurrent API requests...")
start_simulation = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(simulate_user_request) for _ in range(100)]
    results = [future.result() for future in futures]

end_simulation = time.time()
simulation_time = end_simulation - start_simulation

print(f"✅ Completed 100 API requests in {simulation_time:.3f} seconds")
print(f"🚀 API Throughput: {100/simulation_time:.2f} requests/second")

# Display sample results
print("\n📊 Sample API Responses:")
print("-" * 100)
for i, result in enumerate(results[:5]):
    print(f"{i+1}. {result['sentiment']:8s} ({result['confidence']:.3f}) | {result['text'][:60]}...")
    print(f"   ⏱️  Response time: {result['processing_time']*1000:.2f}ms")
    print()

# Get API statistics
api_stats = api.get_stats()
print("📈 Final API Statistics:")
print("-" * 50)
print(f"🔢 Total requests: {api_stats['total_requests']}")
print(f"⏱️  Average response time: {api_stats['avg_response_time']*1000:.2f}ms")
print(f"🚀 Total processing time: {api_stats['total_processing_time']:.3f}s")
print("\n📊 TurboBatcher Statistics:")
batcher_stats = api_stats['batcher_stats']
for key, value in batcher_stats.items():
    if key != 'device':
        print(f"   {key}: {value}")

print(f"\n🎉 TurboBatch handled {api_stats['total_requests']} requests efficiently!")

In [ ]:
# Analyze API performance
api_stats = api.get_stats()

print("📈 API Performance Analysis:")
print("=" * 50)
print(f"Total Requests Processed: {api_stats['total_requests']}")
print(f"Average Response Time: {api_stats['avg_response_time']*1000:.2f}ms")
print(f"Total Processing Time: {api_stats['total_processing_time']:.3f}s")
print(f"API Efficiency: {(api_stats['total_processing_time']/simulation_time)*100:.1f}% CPU utilization")

print("\n🧠 DynamicBatcher Statistics:")
print("-" * 30)
batcher_stats = api_stats['batcher_stats']
for key, value in batcher_stats.items():
    print(f"{key}: {value}")

# Visualize response times
response_times = [r['processing_time'] * 1000 for r in results]  # Convert to ms
sentiments = [r['sentiment'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Response time distribution
ax1.hist(response_times, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(np.mean(response_times), color='red', linestyle='--', 
           label=f'Mean: {np.mean(response_times):.2f}ms')
ax1.set_xlabel('Response Time (ms)')
ax1.set_ylabel('Frequency')
ax1.set_title('⏱️ API Response Time Distribution')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Sentiment distribution
sentiment_counts = {}
for sentiment in sentiments:
    sentiment_counts[sentiment] = sentiment_counts.get(sentiment, 0) + 1

colors = ['#e74c3c' if s == 'NEGATIVE' else '#27ae60' for s in sentiment_counts.keys()]
ax2.pie(sentiment_counts.values(), labels=sentiment_counts.keys(), 
        autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('😊 Sentiment Distribution')

plt.tight_layout()
plt.show()

print(f"\n🎉 API successfully processed {len(results)} requests with an average response time of {np.mean(response_times):.2f}ms!")

## 🔧 Advanced Configuration and Tips

Here are some advanced tips for getting the most out of TurboBatch:

In [ ]:
# Advanced configuration examples
print("🔧 Advanced TurboBatch Configurations:")
print("=" * 50)

configurations = [
    {
        'name': 'High Throughput',
        'config': {
            'max_batch_size': 64,
            'timeout_ms': 200,
            'adaptive_batching': True,
            'performance_monitoring': True
        },
        'use_case': 'Batch processing, offline analysis'
    },
    {
        'name': 'Low Latency',
        'config': {
            'max_batch_size': 8,
            'timeout_ms': 10,
            'adaptive_batching': True,
            'performance_monitoring': False
        },
        'use_case': 'Real-time APIs, interactive applications'
    },
    {
        'name': 'Memory Optimized',
        'config': {
            'max_batch_size': 16,
            'timeout_ms': 100,
            'adaptive_batching': True,
            'performance_monitoring': True
        },
        'use_case': 'Limited GPU memory, large models'
    },
    {
        'name': 'Balanced',
        'config': {
            'max_batch_size': 32,
            'timeout_ms': 50,
            'adaptive_batching': True,
            'performance_monitoring': True
        },
        'use_case': 'General purpose, most applications'
    }
]

for i, config_info in enumerate(configurations, 1):
    print(f"\n{i}. {config_info['name']} Configuration:")
    print(f"   Use Case: {config_info['use_case']}")
    print("   Settings:")
    for key, value in config_info['config'].items():
        print(f"     {key}: {value}")

print("\n💡 Pro Tips:")
tips = [
    "🚀 Use larger batch sizes for offline processing",
    "⚡ Use smaller timeouts for real-time applications", 
    "🧠 Enable adaptive batching for variable workloads",
    "📊 Enable performance monitoring for optimization insights",
    "💾 Monitor GPU memory usage when increasing batch sizes",
    "🔄 Profile your specific use case to find optimal settings"
]

for tip in tips:
    print(f"   {tip}")

print("\n🔥 Example: Creating a high-throughput configuration")
print("-" * 50)

# Example of creating a high-throughput batcher
high_throughput_batcher = TurboBatcher(
    model=model,
    tokenizer=tokenizer,
    max_batch_size=64,
    timeout_ms=200,
    adaptive_batching=True,
    performance_monitoring=True,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print("✅ High-throughput TurboBatcher created!")
print(f"📦 Max batch size: {high_throughput_batcher.max_batch_size}")
print(f"⏱️  Timeout: {high_throughput_batcher.timeout_ms}ms")
print(f"🧠 Adaptive batching: {high_throughput_batcher.adaptive_batching}")

## 🎉 Conclusion

Congratulations! You've successfully learned how to use **TurboBatch** to dramatically improve your transformer inference performance.

### Key Takeaways:

✅ **Performance Boost**: TurboBatch can provide 5-10x speedup over individual processing

✅ **Easy Integration**: Works seamlessly with any HuggingFace transformer model

✅ **Adaptive Intelligence**: Automatically optimizes batch sizes based on workload

✅ **Production Ready**: Built-in monitoring and performance analytics

✅ **Memory Efficient**: Smart memory management prevents OOM errors

### Next Steps:

1. 🔄 **Try it with your own models** - TurboBatch works with any transformer!
2. 📊 **Experiment with configurations** - Find the optimal settings for your use case
3. 🚀 **Deploy to production** - Scale your ML applications with confidence
4. 📈 **Monitor performance** - Use built-in analytics to optimize further

### Resources:

- 📖 **GitHub Repository**: [github.com/Shayanthn/turbobatch](https://github.com/Shayanthn/turbobatch)
- 📚 **Documentation**: Check the README for detailed API reference
- 💬 **Discussions**: Join the community for questions and ideas
- 🐛 **Issues**: Report bugs or request features

### Thank You! 🙏

Thank you for exploring TurboBatch! If this helped accelerate your projects, please:
- ⭐ **Star the repository** on GitHub
- 📢 **Share with colleagues** who work with transformers
- 💡 **Contribute ideas** for improvements

**Happy Accelerating! 🚀**

---

*Created by [Shayan Taherkhani](https://shayantaherkhani.ir) with ❤️*